In [16]:
import requests 
import re 
from pathlib import Path

In [24]:
#Initialize variables that we will use in the data extraction part
genres_dict={'Computers': '48', 'Business': '69', 'Health': '58', 'Romance' :'38', 'Horror': '18'}
username = 'yourname@gmail.com'
password ='yourpassword'
website = 'https://manybooks.net'
login_url = 'https://manybooks.net/mnybks-login-form'
titles_url = 'https://manybooks.net/books/get/' 
genres = ['Computers', 'Business', 'Health', 'Romance', 'Horror']
path = Path('Clasif_data')

In [14]:
#Create folders to divide book files
for folder in genres:
    dest = path/folder
    dest.mkdir(parents=True, exist_ok=True)

In [29]:
#I initiate sessions to get web pages  
session = requests.Session()
#I searched through the website for the parameters to send in order to be accepted when login.
payload = {'__cfduid':cfduid,
           'email':username,
           'pass':password,
           'form_build_id':'form-aIrK-Lvd2CvE8upWr1Yjw_BV7duIw4PSWnfa8HQ4syo',
           'form_id' : 'mb_user_login_form',
           'ga_event' : 'lrf:kZXSGyeM4Zi2Q5n'}

#I donwload 3 pages of books from each of the 5 chosen genres
for genre in genres:
    for i in range(3):#I could loop here through all the pages if my goal was to donwload the complete site
        if i == 0: #The first page when you search for a genre is slightly different
            search = session.get('https://manybooks.net/search-book?field_genre[' + genres_dict[genre] +']=' + genres_dict[genre], headers={ 
                                                'Connection':'keep-alive', 
                                                'Referer': login_url }) #I get the search page to scrape
            soup_string = search.text 
            # I use regex to create a list with all the book IDs. I skip the first 4 because they are recommendations.
            list_of_books = re.findall(r'(?<=node-id=")\d+', soup_string, flags=0)[4:]
            
        else: #The rest of pages of the search have an index at the end of the address so I use the 'i' of the for loop 
            search = session.get('https://manybooks.net/search-book?language=All&field_genre[' + genres_dict[genre] +']='+genres_dict[genre]+'&sticky=All&created_op=%3C&created[value]=0&created[min]=&created[max]=&author_uid_op=%3E%3D&author_uid[value]=0&author_uid[min]=&author_uid[max]=&sort_by=field_downloads&search=&page='+ str(i), headers={ 
                                                'Connection':'keep-alive', 
                                                'Referer': login_url })
            soup_string = search.text 
            list_of_books = list_of_books + re.findall(r'(?<=node-id=")\d+', soup_string, flags=0)[4:]
    session.get(login_url)#I get the login website in order to receive the cookie
    cfduid = session.cookies['__cfduid'] #I save the cookie in a variable to resend it
    session.post(login_url, data=payload, headers={ 'Connection':'keep-alive', #Log in
                                                    'Referer': login_url })
    #I loop though the list of books and donwload them. The '/8' is added to choose the format txt                                             
    for book_id in list_of_books:
        book_url = titles_url + book_id + '/8'#Creating the book url
        r = session.get(book_url,allow_redirects=True, headers={ 'Connection':'keep-alive', #getting the book 
                                                'Referer': 'https://manybooks.net/' })
        with open(str(path)+'/' + genre +'/' + book_id + '.txt', 'wb') as outfile:#Writing the book in a file
             outfile.write(r.content)